To begin with we need to import the models we are testing

In [ ]:

from models import ClassifierWrapper, ClusteringWrapper
from datagen import generate_classification_data, generate_clustering_data


